# Improved Data Aggregation Pipeline

## Key Improvements:
1. ✅ Robust date parsing and validation
2. ✅ Automatic data type conversion
3. ✅ Comprehensive logging and error handling
4. ✅ Data quality checks and validation
5. ✅ Intelligent column mapping
6. ✅ Schema validation across files

In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path
from datetime import datetime
import warnings
from collections import Counter
import re

warnings.filterwarnings('ignore')

# Set display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.width', 1000)

## Configuration

In [12]:
# Configuration
FOLDER_PATH = "../datasets"
OUTPUT_CSV = "../datasets/combined_clean.csv"
OUTPUT_EXCEL = "../merged_data.xlsx"
LOG_FILE = "data_aggregation_log.txt"

# Columns to drop (non-essential)
COLUMNS_TO_DROP = [
    'Balance', 'Remaning', 'Achived', 'Bills',
    'G- Shock', 'G-sale', 'G-profit', 'TChain', 'Tbracelet',
    'Zippo', 'Store Time', 'Unnamed: 15', 'Unnamed: 4', 'Unnamed: 6',
    'Tips: Jay'
]

# Expected date column names (case-insensitive)
DATE_COLUMNS = ['date', 'Date', 'DATE']

# Employee name mappings (handle variations)
EMPLOYEE_MAPPINGS = {
    'dheeraj.1': 'dheeraj_hours',
    'shashi.1': 'shashi_hours',
    'yash.1': 'yash_hours',
    'nithin.1': 'nithin_hours',
    'sunny.1': 'sunny_hours',
    'siddharth.1': 'siddharth_hours',
    'atharv.1': 'atharv_hours'
}

## Helper Functions

In [13]:
def setup_logging(log_file=LOG_FILE):
    """Setup logging to file and console"""
    import logging
    logging.basicConfig(
        level=logging.INFO,
        format='%(asctime)s - %(levelname)s - %(message)s',
        handlers=[
            logging.FileHandler(log_file, mode='w'),
            logging.StreamHandler()
        ]
    )
    return logging.getLogger(__name__)

logger = setup_logging()

In [14]:
def parse_date_flexible(date_value):
    """Parse date with multiple format attempts"""
    if pd.isna(date_value):
        return None
    
    # If already datetime, return as is
    if isinstance(date_value, pd.Timestamp):
        return date_value
    
    # Try common date formats
    date_formats = [
        '%Y-%m-%d',
        '%m/%d/%Y',
        '%d/%m/%Y',
        '%Y-%m-%d %H:%M:%S',
        '%m-%d-%Y',
        '%d-%m-%Y'
    ]
    
    for fmt in date_formats:
        try:
            return pd.to_datetime(date_value, format=fmt)
        except:
            continue
    
    # Last resort: pandas auto-detect
    try:
        return pd.to_datetime(date_value, errors='coerce')
    except:
        return None

In [15]:
def extract_month_year_from_filename(filename):
    """Extract month and year from filename"""
    # Remove extension
    name = Path(filename).stem
    
    # Month name to number mapping
    months = {
        'january': 1, 'jan': 1,
        'february': 2, 'feb': 2,
        'march': 3, 'mar': 3,
        'april': 4, 'apr': 4,
        'may': 5,
        'june': 6, 'jun': 6,
        'july': 7, 'jul': 7,
        'august': 8, 'aug': 8,
        'september': 9, 'sep': 9, 'september': 9,
        'october': 10, 'oct': 10,
        'november': 11, 'nov': 11,
        'december': 12, 'dec': 12
    }
    
    name_lower = name.lower()
    
    # Extract year (4 digits)
    year_match = re.search(r'\b(20\d{2})\b', name)
    year = int(year_match.group(1)) if year_match else None
    
    # Extract month
    month = None
    for month_name, month_num in months.items():
        if month_name in name_lower:
            month = month_num
            break
    
    return month, year

In [16]:
def normalize_column_names(df):
    """Normalize column names: lowercase, strip, handle duplicates"""
    df = df.copy()
    
    # Step 1: Strip and lowercase
    df.columns = df.columns.str.strip().str.lower()
    
    # Step 2: Identify duplicate column names
    counts = Counter(df.columns)
    duplicate_groups = {name: [] for name, count in counts.items() if count > 1}
    
    # Step 3: Group duplicate columns by name and get their positions
    for idx, col_name in enumerate(df.columns):
        if col_name in duplicate_groups:
            duplicate_groups[col_name].append(idx)
    
    # Step 4: Combine duplicate columns and build new dataframe
    combined_data = {}
    processed_duplicates = set()
    
    for idx, col_name in enumerate(df.columns):
        # Skip if this column is a duplicate we've already processed
        if col_name in processed_duplicates:
            continue
            
        if col_name in duplicate_groups:
            # This is a duplicate group - combine them
            duplicate_indices = duplicate_groups[col_name]
            
            # Get all duplicate columns
            duplicate_cols = df.iloc[:, duplicate_indices]
            
            # Combine using backfill (fill missing values from right to left)
            combined = duplicate_cols.bfill(axis=1).iloc[:, 0]
            
            combined_data[col_name] = combined
            
            # Mark all duplicates as processed
            processed_duplicates.add(col_name)
        else:
            # Regular column - keep as is
            combined_data[col_name] = df.iloc[:, idx]
    
    # Step 5: Create new dataframe with unique columns
    result_df = pd.DataFrame(combined_data)
    
    # Step 6: Ensure no remaining duplicates (safety check)
    result_df = result_df.loc[:, ~result_df.columns.duplicated()]
    
    return result_df


In [17]:
def convert_data_types(df):
    """Convert data types intelligently"""
    df = df.copy()
    
    # Parse date column
    date_col = None
    for col in DATE_COLUMNS:
        if col.lower() in df.columns:
            date_col = col.lower()
            break
    
    if date_col:
        df[date_col] = df[date_col].apply(parse_date_flexible)
    
    # Convert numeric columns (employee sales, hours, tips)
    numeric_patterns = [
        'hours', 'tips', 'total', 'jay', 'yash', 'dheeraj',
        'nithin', 'shashi', 'atharv', 'sunny', 'siddharth'
    ]
    
    for col in df.columns:
        col_lower = col.lower()
        
        # Skip date and day columns
        if col_lower in ['date', 'day', 'time', 'source_file', 'source file']:
            continue
        
        # Check if column should be numeric
        if any(pattern in col_lower for pattern in numeric_patterns):
            # Try to convert to numeric
            df[col] = pd.to_numeric(df[col], errors='coerce')
    
    return df

In [18]:
def read_excel_file(file_path, file_name):
    """Read Excel file with error handling and validation"""
    try:
        # Try reading without row limit first to check file size
        df_temp = pd.read_excel(file_path, sheet_name=0, nrows=5)
        
        # Read full file (nrows=32 limit)
        df = pd.read_excel(os.path.join(folder_path, file),nrows=32, sheet_name=0)
        
        # Add metadata
        df['source_file'] = file_name
        
        # Extract month/year from filename
        month, year = extract_month_year_from_filename(file_name)
        if month:
            df['file_month'] = month
        if year:
            df['file_year'] = year
        
        logger.info(f"✅ Successfully read {file_name}: {len(df)} rows, {len(df.columns)} columns")
        
        return df, None
        
    except Exception as e:
        error_msg = f"❌ Failed to read {file_name}: {str(e)}"
        logger.error(error_msg)
        return None, error_msg

In [19]:
def validate_data_quality(df, file_name):
    """Validate data quality and return report"""
    report = {
        'file': file_name,
        'rows': len(df),
        'columns': len(df.columns),
        'missing_dates': 0,
        'missing_totals': 0,
        'duplicate_rows': 0,
        'date_range': None
    }
    
    # Check for date column
    date_col = None
    for col in DATE_COLUMNS:
        if col.lower() in df.columns:
            date_col = col.lower()
            break
    
    if date_col:
        report['missing_dates'] = df[date_col].isna().sum()
        valid_dates = df[date_col].dropna()
        
        if len(valid_dates) > 0:
            # Convert to datetime if needed (handle mixed types)
            try:
                # Try to convert to datetime if not already
                if not pd.api.types.is_datetime64_any_dtype(valid_dates):
                    valid_dates = pd.to_datetime(valid_dates, errors='coerce')
                
                # Only compute min/max if we have valid datetime values
                valid_datetime = valid_dates.dropna()
                if len(valid_datetime) > 0:
                    report['date_range'] = (valid_datetime.min(), valid_datetime.max())
            except Exception as e:
                # If conversion fails, skip date range
                logger.warning(f"Could not compute date range for {file_name}: {e}")
                report['date_range'] = None
    
    # Check for total column
    if 'total' in df.columns:
        report['missing_totals'] = df['total'].isna().sum()
    
    # Check for duplicates
    if date_col:
        report['duplicate_rows'] = df.duplicated(subset=[date_col]).sum()
    
    return report


## Main Aggregation Pipeline

In [20]:
# Get all Excel files
folder_path = Path(FOLDER_PATH)
all_files = sorted([f for f in os.listdir(folder_path) if f.endswith(".xlsx")])

logger.info(f"📁 Found {len(all_files)} Excel files to process")
print(f"\n📁 Found {len(all_files)} Excel files to process\n")

2026-01-28 09:21:29,712 - INFO - 📁 Found 32 Excel files to process



📁 Found 32 Excel files to process



In [21]:
# Process all files
dfs = []
errors = []
quality_reports = []

for file in all_files:
    file_path = folder_path / file
    
    # Read file
    df, error = read_excel_file(file_path, file)
    
    if df is None:
        errors.append({'file': file, 'error': error})
        continue
    
    # Normalize columns
    df = normalize_column_names(df)
    
    # Validate data quality
    quality_report = validate_data_quality(df, file)
    quality_reports.append(quality_report)
    
    # Convert data types
    df = convert_data_types(df)
    
    # Apply employee column mappings
    df = df.rename(columns=EMPLOYEE_MAPPINGS)
    
    dfs.append(df)

logger.info(f"\n✅ Successfully processed {len(dfs)} files")
logger.info(f"❌ Failed to process {len(errors)} files")

print(f"\n✅ Successfully processed {len(dfs)} files")
print(f"❌ Failed to process {len(errors)} files\n")

2026-01-28 09:21:30,123 - INFO - ✅ Successfully read April 2025.xlsx: 32 rows, 24 columns
2026-01-28 09:21:30,155 - INFO - ✅ Successfully read April24.xlsx: 30 rows, 25 columns
2026-01-28 09:21:30,182 - INFO - ✅ Successfully read August 2023.xlsx: 31 rows, 23 columns
2026-01-28 09:21:30,209 - INFO - ✅ Successfully read August 2025.xlsx: 31 rows, 24 columns
2026-01-28 09:21:30,418 - INFO - ✅ Successfully read August 24.xlsx: 31 rows, 26 columns
2026-01-28 09:21:30,444 - INFO - ✅ Successfully read December 2023.xlsx: 31 rows, 25 columns
2026-01-28 09:21:30,567 - INFO - ✅ Successfully read December 2025.xlsx: 31 rows, 17 columns
2026-01-28 09:21:30,597 - INFO - ✅ Successfully read December2022.xlsx: 31 rows, 24 columns
2026-01-28 09:21:30,717 - INFO - ✅ Successfully read December2024.xlsx: 31 rows, 26 columns
2026-01-28 09:21:30,741 - INFO - ✅ Successfully read Feb 2023.xlsx: 31 rows, 23 columns
2026-01-28 09:21:30,769 - INFO - ✅ Successfully read Feb2024.xlsx: 32 rows, 24 columns
2026-01


✅ Successfully processed 32 files
❌ Failed to process 0 files



In [22]:
# Combine all dataframes
if dfs:
    combined_df = pd.concat(dfs, ignore_index=True)
    logger.info(f"📊 Combined dataset: {len(combined_df)} rows, {len(combined_df.columns)} columns")
    print(f"\n📊 Combined dataset: {len(combined_df)} rows, {len(combined_df.columns)} columns\n")
else:
    logger.error("No dataframes to combine!")
    raise ValueError("No dataframes to combine!")

2026-01-28 09:21:33,093 - INFO - 📊 Combined dataset: 995 rows, 41 columns



📊 Combined dataset: 995 rows, 41 columns



In [23]:
# Drop unnecessary columns (case-insensitive)
columns_to_drop_lower = [col.lower() for col in COLUMNS_TO_DROP]
existing_cols_to_drop = [col for col in combined_df.columns if col.lower() in columns_to_drop_lower]

if existing_cols_to_drop:
    combined_df = combined_df.drop(columns=existing_cols_to_drop)
    logger.info(f"🗑️  Dropped {len(existing_cols_to_drop)} unnecessary columns")
    print(f"🗑️  Dropped {len(existing_cols_to_drop)} unnecessary columns")

# Final column normalization
combined_df = normalize_column_names(combined_df)

2026-01-28 09:21:33,101 - INFO - 🗑️  Dropped 15 unnecessary columns


🗑️  Dropped 15 unnecessary columns


In [24]:
# Display summary
print("\n" + "="*60)
print("DATA SUMMARY")
print("="*60)
print(f"Total Rows: {len(combined_df)}")
print(f"Total Columns: {len(combined_df.columns)}")
print(f"\nColumn Names:")
for i, col in enumerate(combined_df.columns, 1):
    print(f"  {i:2d}. {col}")

print("\n" + "="*60)
print("DATA TYPES")
print("="*60)
print(combined_df.dtypes)

print("\n" + "="*60)
print("MISSING VALUES")
print("="*60)
missing = combined_df.isnull().sum()
missing_pct = (missing / len(combined_df) * 100).round(2)
missing_df = pd.DataFrame({
    'Column': missing.index,
    'Missing Count': missing.values,
    'Missing %': missing_pct.values
})
missing_df = missing_df[missing_df['Missing Count'] > 0].sort_values('Missing Count', ascending=False)
print(missing_df.to_string(index=False))


DATA SUMMARY
Total Rows: 995
Total Columns: 26

Column Names:
   1. date
   2. day
   3. time
   4. jay
   5. dheeraj
   6. shashi
   7. total
   8. dheeraj_hours
   9. shashi_hours
  10. tips: shashi
  11. tips: dheeraj
  12. source_file
  13. file_month
  14. file_year
  15. yash
  16. nithin
  17. yash_hours
  18. total_employees_hours
  19. total_employees
  20. nithin_hours
  21. tips: yash
  22. sunny
  23. sunny_hours
  24. siddharth
  25. atharv
  26. atharv_hours

DATA TYPES
date                     datetime64[ns]
day                              object
time                             object
jay                             float64
dheeraj                         float64
shashi                          float64
total                           float64
dheeraj_hours                   float64
shashi_hours                    float64
tips: shashi                    float64
tips: dheeraj                   float64
source_file                      object
file_month                    

In [25]:
# Display quality reports
if quality_reports:
    print("\n" + "="*60)
    print("DATA QUALITY REPORT BY FILE")
    print("="*60)
    
    quality_df = pd.DataFrame(quality_reports)
    print(quality_df.to_string(index=False))
    
    # Summary statistics
    print("\n" + "="*60)
    print("SUMMARY STATISTICS")
    print("="*60)
    print(f"Total rows across all files: {quality_df['rows'].sum()}")
    print(f"Average rows per file: {quality_df['rows'].mean():.1f}")
    print(f"Files with missing dates: {(quality_df['missing_dates'] > 0).sum()}")
    print(f"Files with missing totals: {(quality_df['missing_totals'] > 0).sum()}")
    print(f"Files with duplicate rows: {(quality_df['duplicate_rows'] > 0).sum()}")


DATA QUALITY REPORT BY FILE
                file  rows  columns  missing_dates  missing_totals  duplicate_rows                                 date_range
     April 2025.xlsx    32       24              2               1               1 (2025-04-01 00:00:00, 2025-04-30 00:00:00)
        April24.xlsx    30       25              0               0               0 (2024-04-01 00:00:00, 2024-04-30 00:00:00)
    August 2023.xlsx    31       22              0               0               0 (2023-08-01 00:00:00, 2023-08-31 00:00:00)
    August 2025.xlsx    31       24              1               0               0 (2025-08-01 00:00:00, 2025-08-30 00:00:00)
      August 24.xlsx    31       25              0               0               0 (2024-08-01 00:00:00, 2024-08-31 00:00:00)
  December 2023.xlsx    31       24              0               1               0 (2023-12-01 00:00:00, 2023-12-31 00:00:00)
  December 2025.xlsx    31       17              0               0               0 (2025-

In [26]:
# Display error report
if errors:
    print("\n" + "="*60)
    print("ERROR REPORT")
    print("="*60)
    for error in errors:
        print(f"\nFile: {error['file']}")
        print(f"Error: {error['error']}")

In [27]:
# Save outputs
print("\n" + "="*60)
print("SAVING OUTPUTS")
print("="*60)

# Save to CSV
combined_df.to_csv(OUTPUT_CSV, index=False)
logger.info(f"💾 Saved to CSV: {OUTPUT_CSV}")
print(f"💾 Saved to CSV: {OUTPUT_CSV}")

# Save to Excel
combined_df.to_excel(OUTPUT_EXCEL, index=False)
logger.info(f"💾 Saved to Excel: {OUTPUT_EXCEL}")
print(f"💾 Saved to Excel: {OUTPUT_EXCEL}")

print("\n✅ Data aggregation complete!")
logger.info("✅ Data aggregation complete!")

2026-01-28 09:21:33,247 - INFO - 💾 Saved to CSV: ../datasets/combined_clean.csv



SAVING OUTPUTS
💾 Saved to CSV: ../datasets/combined_clean.csv


2026-01-28 09:21:33,423 - INFO - 💾 Saved to Excel: ../merged_data.xlsx
2026-01-28 09:21:33,424 - INFO - ✅ Data aggregation complete!


💾 Saved to Excel: ../merged_data.xlsx

✅ Data aggregation complete!


In [28]:
# Preview the final dataset
print("\n" + "="*60)
print("PREVIEW OF COMBINED DATA")
print("="*60)
print(combined_df.head(10))

print("\n" + "="*60)
print("DATA INFO")
print("="*60)
combined_df.info()


PREVIEW OF COMBINED DATA
        date        day     time    jay  dheeraj  shashi  total  dheeraj_hours  shashi_hours  tips: shashi  tips: dheeraj      source_file  file_month  file_year  yash  nithin  yash_hours  total_employees_hours  total_employees  nithin_hours  tips: yash  sunny  sunny_hours  siddharth  atharv  atharv_hours
0 2025-04-01    Tuesday  11 TO 8  306.0      NaN     NaN  306.0            NaN           NaN           NaN            NaN  April 2025.xlsx           4     2025.0   NaN     NaN         NaN                    NaN              NaN           NaN         NaN    NaN          NaN        NaN     NaN           NaN
1 2025-04-02  Wednesday  11 TO 8    NaN    117.0     NaN  117.0            5.0           NaN           NaN            NaN  April 2025.xlsx           4     2025.0   NaN     NaN         NaN                    NaN              NaN           NaN         NaN    NaN          NaN        NaN     NaN           NaN
2 2025-04-03   Thursday  11 TO 8  405.0      NaN     